In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
import os

# Kafka settings
kafka_topic_name = "stack_exchange"
kafka_bootstrap_servers = "localhost:39092"

In [3]:
spark_jars = "{},{},{},{}".format(
    os.getcwd() + "/jars/spark-sql-kafka-0-10_2.12-3.5.0.jar",
    os.getcwd() + "/jars/kafka-clients-3.4.1.jar",
    os.getcwd() + "/jars/commons-pool2-2.11.1.jar",
    os.getcwd() + "/jars/spark-token-provider-kafka-0-10_2.12-3.5.0.jar",
)

# Spark session & context
spark = (
    SparkSession.builder.master("local")
    .appName("KafkaSparkStreaming")
    .config("spark.jars", spark_jars)
    .getOrCreate()
)

In [4]:
# Create stream dataframe setting kafka server, topic, and offset option
df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers)  # Kafka server
    .option("subscribe", kafka_topic_name)  # Topic
    .load()
)

In [5]:
query = (
    df.selectExpr("CAST(value AS STRING)")
    .writeStream.outputMode("append")
    .format("console")
    .start()
)

# Chờ query kết thúc
query.awaitTermination()